# Some Key Concepts about PCA Implementation

For large datasets, computing the covariance matrix and its eigenvalues/eigenvectors can be computationally expensive and inefficient. This inefficiency arises from the quadratic complexity involved in calculating the full covariance matrix with respect to the number of features. In contrast, PCA is typically implemented in a more optimized manner, making it better suited for handling large datasets efficiently.

The key distinction between the two approaches lies in the computation of principal components. PCA directly computes a maximum of `min(n_samples, n_features)` components, avoiding the need to compute the full covariance matrix.

We can demonstrate that both approaches yield the same principal components through a simple example:

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

# Generate a random data matrix X (5 samples, 30 features)
# Each row in X corresponds to a sample and each column corresponds to a feature.
X = np.random.rand(5, 30)

# Compute the covariance matrix of the data
# The covariance matrix describes the variance and correlation between each pair of features.
cov_matrix = np.cov(X.T)

# Compute eigenvalues and eigenvectors of the covariance matrix
# Eigenvalues represent the variance along each eigenvector (principal component),
# and the eigenvectors are the directions of maximum variance.
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

# Sort eigenvalues and eigenvectors in descending order of eigenvalues
# We want the eigenvectors corresponding to the largest eigenvalues, as they represent the most important principal components.
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

# Print the shapes of the covariance matrix, eigenvalues, and eigenvectors
print("Shape of covariance matrix:", cov_matrix.shape)  # Should be (30, 30)
print("Shape of eigenvalues:", eigenvalues.shape)        # Should be (30,)
print("Shape of eigenvectors:", eigenvectors.shape)      # Should be (30, 30)

In [ ]:
# Initialize PCA
# We fit PCA to use it directly on the data instead of the covariance matrix.
# PCA will automatically compute the eigenvalues and eigenvectors for us.
pca = PCA()
pca.fit(X)

# Extract principal components (eigenvectors sorted by explained variance)
principal_components = pca.components_

# Print the shape of the principal components (similar to eigenvectors)
print("Shape of principal_components:", principal_components.shape)  # Should be (n_components, 30)

In [ ]:
# Compare the first eigenvector from the covariance matrix with the first principal component
# The eigenvectors and the principal components are related but may differ by sign and order.
print("Eigenvector from covariance matrix (1st):", eigenvectors[:, 1])
print("Principal Component (1st):", principal_components[1])

# Check if the first eigenvector and principal component are the same (up to a sign flip)
# They should be the same (ignoring sign) because PCA uses eigenvectors of the covariance matrix.
print("Are the first eigenvector and principal component the same (up to sign)?")
print(np.allclose(eigenvectors[:, 0], principal_components[0]))

# Statistical Shape Modeling in Biomechanics: Heart Left Ventricle

In this notebook, we will learn how to build a a statistical shape model (SSM) of the human left ventricle using Python libraries. The notebook is based on the GitHub repository [Statistical-Shape-Model](https://github.com/UK-Digital-Heart-Project/Statistical-Shape-Model) built within the UK Digital Heart Project, which was set up by Prof. Stuart Cook and Dr. Declan O'Regan at the MRC Clinical Sciences Centre, Imperial College London, UK.


# Preprocessing




In [ ]:
# Import necessary libraries
import os  # For interacting with the file system
import shutil

In [ ]:
# Setting up the environment
os.chdir('/content')

# Check if the folder 'NotebookROM' exists before attempting to delete it to reinstall it
if os.path.exists('NotebookSSM'):
    # Remove the folder and all its contents
    shutil.rmtree('NotebookSSM')

# Pull the repo with data and install packages for the notebook if we're in Google Colab
try:
  import google.colab # Check if we're in Google Colab
  !sudo apt install libgl1-mesa-glx xvfb
  !pip install pyvista[jupyter] -qq
  current_directory = os.path.basename(os.getcwd())
  # If we're not in the right directory or the directory doesn't exist, clone the repo
  if not (os.path.isdir('NotebookSSM') or current_directory == 'NotebookSSM'):
    !git clone https://github.com/bisighinibeatrice/NotebookSSM.git
  if os.path.isdir('NotebookSSM'):
    %cd NotebookSSM
except:
    pass

In [ ]:
# Import necessary libraries
import vtk  # Library for handling VTK files
import numpy as np  # For numerical computations and array manipulations
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

The SSM will be constructed using 100 shapes (`M = 100`). These shapes were already registered to the template space using rigid registration (so that the position and orientation differences are removed).

In the following code, we will read all the shapes that are contained in the `Input` folder and store them in the matrix `X`.

> **NOTA BENE:** Contrary from what seen in the theoretical part of the class, the matrix `X` should be organized such that rows represent different shapes (samples) and columns represent features. This is requested by the `PCA` function.

In [ ]:
# Directory containing the VTK files
vtk_dir = 'Input/'  # Set the appropriate directory where your VTK files are saved

# List all VTK files in the directory that include 'sample' in their filenames
vtk_files = [f for f in os.listdir(vtk_dir) if f.endswith('.vtk') and 'sample' in f]

# Print the number of VTK files found in the specified directory
print(f"Number of VTK files found: {len(vtk_files)}")

# Initialize a list to store all shapes for subsequent analysis (PCA)
shapes = []

# Process each VTK file: read the data and extract point coordinates
for file in vtk_files:
    reader = vtk.vtkPolyDataReader()  # Create a reader to handle VTK file format
    reader.SetFileName(os.path.join(vtk_dir, file))  # Specify the file to read
    reader.Update()  # Parse the file and load data into the reader

    # Extract the geometry (polydata) from the file
    polydata = reader.GetOutput()
    points = polydata.GetPoints()  # Access the points in the polydata
    n_points = points.GetNumberOfPoints()  # Count the number of points in the dataset

    # Flatten the 3D coordinates of all points into a single vector
    shape = []  # Temporary storage for current shape's flattened coordinates
    for i in range(n_points):
        shape.extend(points.GetPoint(i))  # Add [x, y, z] of each point to the shape vector

    # Append the flattened shape vector to the list of all shapes
    shapes.append(shape)

# Convert the list of shapes into a NumPy matrix (2D array)
X = np.array(shapes)

How many features is each shape composed of (or equivalently, how much is `N`)? How many points compose each shape?

In [ ]:
# Compute the number of features
print(f"Number of features (coordinates): {X.shape[1]}")
print(f"Number of points: {X.shape[1]//3}")

# Visualisation

We can visualise these shapes as point-clouds using `Plotly` for interactive 3D visualization.

In [ ]:
# Function to read points from a VTK file
def read_vtk_points(vtk_file_path):
    """
    Read points (vertices) from a VTK file.

    Parameters:
        vtk_file_path (str): Full path to the VTK file.

    Returns:
        numpy.ndarray: Array of shape (n_points, 3) containing the 3D coordinates.
    """
    # Read the VTK file
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(vtk_file_path)
    reader.Update()

    # Extract points (vertices) from the polydata
    polydata = reader.GetOutput()
    points = polydata.GetPoints()
    n_points = points.GetNumberOfPoints()

    # Convert points to a numpy array
    numpy_points = np.array([points.GetPoint(i) for i in range(n_points)])

    return numpy_points

# Function to visualize multiple VTK meshes interactively using Plotly
def visualize_vtk_meshes_plotly(mesh_points_list, mesh_names):
    """
    Visualize the meshes interactively using Plotly.

    Parameters:
        mesh_points_list (list of numpy.ndarray): List of numpy arrays, each containing the points for a mesh.
        mesh_names (list of str): List of names for each mesh (e.g., file names).
    """
    fig = go.Figure()

    for i, points in enumerate(mesh_points_list):
        # Add the points from this mesh as a trace in the plot
        fig.add_trace(go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode='markers',
            marker=dict(size=2, opacity=0.8),
            name=mesh_names[i]  # Use the file name as the legend
        ))

    # Set plot layout
    fig.update_layout(
        title="3D Shape Visualization (Superimposed)",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z"
        ),
        margin=dict(l=0, r=0, b=0, t=30)
    )

    # Show the plot
    fig.show()

We can read one of the point clouds using `read_vtk_points` and visualize it with `visualize_vtk_meshes_plotly`.

In [ ]:
# Example: Read points from VTK file
vtk_file_paths = [
    os.path.join(vtk_dir, "LV_ED_sample_1.vtk")
]

# Read points from all VTK files
mesh_points_list = [read_vtk_points(file_path) for file_path in vtk_file_paths]
mesh_names = [os.path.basename(file_path) for file_path in vtk_file_paths]

# Visualize the meshes superimposed
print(f"Visualizing files: {', '.join(mesh_names)}")
visualize_vtk_meshes_plotly(mesh_points_list, mesh_names)

We can also superimpose two point clouds.

In [ ]:
# Example: Read points from multiple VTK files
vtk_file_paths = [
    os.path.join(vtk_dir, "LV_ED_sample_55.vtk"),
    os.path.join(vtk_dir, "LV_ED_sample_71.vtk"),
]

# Read points from all VTK files
mesh_points_list = [read_vtk_points(file_path) for file_path in vtk_file_paths]
mesh_names = [os.path.basename(file_path) for file_path in vtk_file_paths]

# Visualize the meshes superimposed
print(f"Visualizing files: {', '.join(mesh_names)}")
visualize_vtk_meshes_plotly(mesh_points_list, mesh_names)

# Principal Component Analysis

The PCA identifies the principal components of the data by analyzing its structure and finding the directions of maximum variance.

If the data is not centered (i.e., it has a non-zero mean), the analysis may reflect the overall offset instead of the true patterns of variation. By centering the data (subtracting the mean), we ensure that the PCA focuses on the intrinsic variation in the data, ignoring any global shifts. This allows  the PCA to identify the directions of maximum variability effectively.

We can compute the mean shape within the database.

In [ ]:
# Compute the mean shape (mean vector across all samples)
X_mean  = np.mean(X, axis=0)
print(f"X_mean dimensions: {X_mean .shape}")

We can cen all the shapes with respect to the computed mean shape.

In [ ]:
# Subtract the mean shape from all shapes (center the data)
X_centered  = X - X_mean
print(f"X_centered dimensions: {X_centered.shape}")

We can now build the PCA object and fit it with our data.

In [ ]:
# Initialize PCA
pca = PCA()
# Fit the model to the data
pca.fit(X_centered)

The principal modes (or components) represent the directions in which the data varies most. They are stored in the `components_` attribute of the fitted PCA object. Each row of `components_` corresponds to a principal mode (or component).

In [ ]:
# Extract the principal modes (components)
principal_components = pca.components_.T  # Transpose to match point-wise interpretation

How many principal compoenent do we have? What is the dimension of  each principal components?

In [ ]:
# Print the dimensions of each principal component
print("Number of principal components:", principal_components.shape[1])
print("Dimension of each principal component:", principal_components.shape[0])

We can plot the cumulative variance explained by each principal component in PCA, you can use the `explained_variance_ratio_` attribute from the fitted PCA model. This attribute contains the proportion of variance explained by each principal component. The cumulative variance can be computed by taking the cumulative sum of these values (`cumsum`).

In [ ]:
# Get explained variance ratio (how much variance each principal component explains)
explained_variance = pca.explained_variance_ratio_

# Compute cumulative variance
cumulative_variance = np.cumsum(explained_variance)

# Plot cumulative variance
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', color='b')
plt.xlabel("Index of Principal Components")
plt.ylabel("Cumulative Explained Variance")
plt.grid(True)
plt.show()

To determine how many principal components are needed to explain 90%, 95%, and 99% of the variance, we can use the cumulative variance plot and check at which point the cumulative variance exceeds these thresholds.


In [ ]:
# Find the number of components for 90%, 95%, and 99% variance explained
components_90 = np.argmax(cumulative_variance >= 0.90) + 1  # +1 because index is zero-based
components_95 = np.argmax(cumulative_variance >= 0.95) + 1  # +1 because index is zero-based
components_99 = np.argmax(cumulative_variance >= 0.99) + 1  # +1 because index is zero-based

# Print the results
print(f"Number of components to explain 90% variance: {components_90}")
print(f"Number of components to explain 95% variance: {components_95}")
print(f"Number of components to explain 99% variance: {components_99}")

In [ ]:
# Plot cumulative variance for visualization
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', color='b')
plt.axvline(x=components_90, color='r', linestyle='--', label="90% Variance")
plt.axvline(x=components_95, color='g', linestyle='--', label="95% Variance")
plt.axvline(x=components_99, color='y', linestyle='--', label="99% Variance")
plt.title("Cumulative Explained Variance")
plt.xlabel("Number of Principal Components")
plt.ylabel("Cumulative Explained Variance")
plt.legend()
plt.grid(True)
plt.show()

After fitting a PCA model to your data, we can compute the coefficients by using the transform method of the PCA object. This method projects the original data onto the principal components, effectively computing the coefficients (i.e., the contributions of each principal component for each data point).

In [ ]:
# Get the coefficients (also known as the projection onto the principal components)
principal_coefficients = pca.transform(X_centered)

# Print the shape of the coefficients matrix
print("Shape of coefficients matrix:", principal_coefficients.shape)

# Reconstruction
Now, we can reconstruct one of the original shapes by using all the principal components obtained from PCA. The idea is to combine these components with their respective weights to approximate the original shape as closely as possible.

Once we have the reconstructed shape, we compare it to the original shape to measure how well the reconstruction captures the original details. To do this, we use the Mean Squared Error (MSE). The MSE quantifies the average squared difference between the original and reconstructed values.

A lower MSE indicates a better reconstruction, meaning the reconstructed shape is very close to the original. If the MSE is high, it suggests that some important details might have been lost during the reconstruction. This process helps us evaluate how effective PCA is at preserving the essential features of the data.

We can start by using all the principal components.

In [ ]:
# Use the first shape for reconstruction
first_shape = X[0]

# Reconstruct the first shape using the all the components
reconstructed_shape = np.dot(principal_coefficients[0,], principal_components[:, :].T) + X_mean

# Compute the Mean Squared Error (MSE) for the reconstruction
mse_error = np.mean((first_shape - reconstructed_shape) ** 2)
print("\nReconstruction Error:")
print(mse_error)

We can now reconstruct the first shape using only a subset of the principal components, rather than all of them.

In [ ]:
# Number of principal components to use
num_components_to_use = 10

# Use the first shape for reconstruction
first_shape = X[0]

# Reconstruct the first shape using only the selected number of components
reconstructed_shape = (
    np.dot(principal_coefficients[0, :num_components_to_use], principal_components[:, :num_components_to_use].T) + X_mean
)

# Compute the Mean Squared Error (MSE) for the reconstruction
mse_error = np.mean((first_shape - reconstructed_shape) ** 2)
print("\nReconstruction Error using", num_components_to_use, "components:")
print(mse_error)

We can modify the code to iteratively increase the number of principal components used for reconstruction, starting from 0 and incrementing by steps of 10, up to the maximum number of components. For each iteration, we can compute and record the reconstruction error (using Mean Squared Error, MSE) to observe how the error decreases as more components are included.

In [ ]:
# Initialize a list to store MSE values
mse_errors = []

# Loop over 1, 5, and 10 principal components to reconstruct the shape
for n_components in range(1, 99, 10):  # n_components ranges from 1 to 50 with step 5
    # Reconstruct the first shape using the first n_components
    reconstructed_shape = np.dot(principal_coefficients[0, :n_components], principal_components[:, :n_components].T) + X_mean

    # Compute the Mean Squared Error (MSE) for the reconstruction
    mse_error = np.mean((first_shape - reconstructed_shape) ** 2)

     # Store the MSE for plotting
    mse_errors.append(mse_error)

    # Print the MSE for the current reconstruction
    print(f"\nReconstruction Error with {n_components} components:")
    print(f"MSE: {mse_error}")

# Plotting the reconstruction errors
plt.figure(figsize=(8, 6))
plt.plot(range(1, 99, 10), mse_errors, marker='o', linestyle='-', color='b', label='Reconstruction Error (MSE)')
plt.title('Reconstruction Error vs. Number of Principal Components')
plt.xlabel('Number of Principal Components')
plt.ylabel('Mean Squared Error (MSE)')
plt.grid(True)
plt.legend()
plt.show()

We can reconstructs the shapes using a specified number of principal components and then superimose the original and reconstructed shapes.

In [ ]:
# Define the shape index and PCA settings
which_shape = 0
n_components = 5

# Read the original shape from the VTK file
original_shape_flat = X[which_shape]  # Access the shape data (e.g., X[0] for the first shape)

# Reshape the original shape into a 3D point format (assuming 3D points: x, y, z)
n_points = original_shape_flat.shape[0] // 3  # Assuming each shape has 3D points (x, y, z)
original_shape = original_shape_flat.reshape(n_points, 3)

# Perform PCA reconstruction using the selected number of components
reconstructed_shape_flat = np.dot(principal_coefficients[which_shape, :n_components], principal_components[:, :n_components].T) + X_mean

# Reshape the reconstructed data back into a 3D point format
n_points = reconstructed_shape_flat.shape[0] // 3  # Reshape back to 3D coordinates
reconstructed_shape = reconstructed_shape_flat.reshape(n_points, 3)

# Compute MSE between original (from VTK) and reconstructed shape
mse_error = np.mean((original_shape_flat - reconstructed_shape_flat) ** 2)
print(f"\nReconstruction Error (MSE) using {n_components} components:")
print(mse_error)

# Visualize original and reconstructed meshes
mesh_points_list = [original_shape, reconstructed_shape]
mesh_names = ["Original Shape", "Reconstructed Shape"]
visualize_vtk_meshes_plotly(mesh_points_list, mesh_names)

# Modes explanation
Each mode represents a different direction of variation in the shape, and by adjusting the strength of this variation, we can see how the shape changes.

In [ ]:
selected_mode = 0

# Read the original shape from the VTK file
mean_shape_flat = X_mean

# Reshape the original shape into a 3D point format (assuming 3D points: x, y, z)
n_points = mean_shape_flat.shape[0] // 3  # Assuming each shape has 3D points (x, y, z)
mean_shape = mean_shape_flat.reshape(n_points, 3)

# Calculate the standard deviation of the coefficients for the selected mode
# principal_coefficients should be the coefficients from the PCA (this is usually X @ principal_components)
mode_std = np.std(principal_coefficients[:, selected_mode])  # Standard deviation of the coefficients for the selected mode
print(f"Standard deviation of the selected mode: {mode_std}")

# Define the coefficient to multiply by the standard deviation to adjust the strength of the modification
coeff = 3.0  # Coefficient for scaling the standard deviation (this is user-controlled)

# Perform PCA reconstruction using the selected number of components
effect_selcted_mode_pos_flat = X_mean + coeff * mode_std * principal_components[:, selected_mode] # Scale the mode's effect
effect_selcted_mode_neg_flat = X_mean - coeff * mode_std * principal_components[:, selected_mode]  # Scale the mode's effect

# Reshape the reconstructed data back into a 3D point format
n_points = effect_selcted_mode_pos_flat.shape[0] // 3  # Reshape back to 3D coordinates
effect_selcted_mode_pos = effect_selcted_mode_pos_flat.reshape(n_points, 3)
n_points = effect_selcted_mode_neg_flat.shape[0] // 3  # Reshape back to 3D coordinates
effect_selcted_mode_neg = effect_selcted_mode_neg_flat.reshape(n_points, 3)

# Visualize original and reconstructed meshes
mesh_points_list = [original_shape, effect_selcted_mode_pos, effect_selcted_mode_neg]
mesh_names = ["Original Shape", "Positive Effect", "Negative Effect"]
visualize_vtk_meshes_plotly(mesh_points_list, mesh_names)


![Modes](https://i.imgur.com/l9IN8NQ.png)

![Modes](https://i.imgur.com/qLT9EgK.png)
